In [1]:
from claude_agent_graph import AgentGraph

# Create a graph
graph = AgentGraph(name="my_network")

# Add nodes (agents)
await graph.add_node(
    node_id="supervisor",
    system_prompt="You coordinate worker agents.",
    model="claude-sonnet-4-20250514"
)

await graph.add_node(
    node_id="worker_1",
    system_prompt="You execute tasks.",
    model="claude-sonnet-4-20250514"
)

# Add edge (connection)
await graph.add_edge(
    from_node="supervisor",
    to_node="worker_1",
    directed=True
)

Edge(edge_id='supervisor_to_worker_1', from_node='supervisor', to_node='worker_1', directed=True, created_at=datetime.datetime(2025, 10, 26, 6, 23, 10, 769909, tzinfo=datetime.timezone.utc), properties={})

In [2]:
# Send message
await graph.send_message(
    from_node="supervisor",
    to_node="worker_1",
    content="Come up with 3 ideas for a new product."
)

Message(message_id='msg_962b1e2ee63844bb', timestamp=datetime.datetime(2025, 10, 26, 6, 23, 21, 365701, tzinfo=datetime.timezone.utc), from_node='supervisor', to_node='worker_1', role=<MessageRole.USER: 'user'>, content='Come up with 3 ideas for a new product.', metadata={})

In [3]:
# Get worker_1 status and recent conversation
print("=== Worker_1 Status ===")
status = graph.get_agent_status("worker_1")
print(f"Status: {status['status']}")
print(f"Model: {status['model']}")
print(f"Is Running: {status['is_running']}")

# Get recent messages between supervisor and worker_1
print("\n=== Recent Messages ===")
messages = await graph.get_recent_messages(
    from_node="supervisor",
    to_node="worker_1",
    count=5
)

for msg in messages:
    print(f"\n[{msg.timestamp}] {msg.from_node} -> {msg.to_node}:")
    print(f"Content: {msg.content}")
    if msg.metadata:
        print(f"Metadata: {msg.metadata}")

=== Worker_1 Status ===
Status: active
Model: claude-sonnet-4-20250514
Is Running: True

=== Recent Messages ===

[2025-10-26 06:23:21.365701+00:00] supervisor -> worker_1:
Content: Come up with 3 ideas for a new product.


In [7]:
# Examine worker_1's internal state and conversation history
print("=== Worker_1 Internal Examination ===\n")

# Get the worker_1 node object
worker_node = graph.get_node("worker_1")
print(f"Node ID: {worker_node.node_id}")
print(f"System Prompt: {worker_node.system_prompt}")
print(f"Model: {worker_node.model}")
print(f"Original Prompt: {worker_node.original_system_prompt}")
print(f"Effective Prompt: {worker_node.effective_system_prompt}")
print(f"Prompt Dirty Flag: {worker_node.prompt_dirty}")
print(f"Metadata: {worker_node.metadata}")

# Get all incoming edges (messages TO worker_1)
print("\n=== Incoming Edges (Messages TO worker_1) ===")
incoming = graph.get_neighbors("worker_1", direction="incoming")
print(f"Sources: {incoming}")

# Get all outgoing edges (messages FROM worker_1)
print("\n=== Outgoing Edges (Messages FROM worker_1) ===")
outgoing = graph.get_neighbors("worker_1", direction="outgoing")
print(f"Destinations: {outgoing}")

# Get all conversations involving worker_1
print("\n=== All Conversations Involving worker_1 ===")
supervisor_to_worker = await graph.get_conversation(
    from_node="supervisor",
    to_node="worker_1"
)
print(f"Messages from supervisor to worker_1: {len(supervisor_to_worker)}")
for msg in supervisor_to_worker:
    print(f"  [{msg.timestamp}] {msg.from_node} -> {msg.to_node}: {msg.content[:50]}...")

# Check if worker_1 has sent any messages back to supervisor
worker_to_supervisor = await graph.get_conversation(
    from_node="worker_1",
    to_node="supervisor"
)
print(f"\nMessages from worker_1 to supervisor: {len(worker_to_supervisor)}")
for msg in worker_to_supervisor:
    print(f"  [{msg.timestamp}] {msg.from_node} -> {msg.to_node}: {msg.content[:50]}...")

# Get all controllers of worker_1 (who supervises it)
print("\n=== Control Relationships ===")
controllers = graph.get_controllers("worker_1")
print(f"Controllers of worker_1: {controllers}")

subordinates = graph.get_subordinates("worker_1")
print(f"Subordinates of worker_1: {subordinates}")

=== Worker_1 Internal Examination ===

Node ID: worker_1
System Prompt: You execute tasks.
Model: claude-sonnet-4-20250514
Original Prompt: None
Effective Prompt: You execute tasks.

## Control Hierarchy
You are agent 'worker_1'. You report to the following controllers:
  - Agent 'supervisor' (supervisor)

Follow directives from your controllers while maintaining your specialized role.
Prompt Dirty Flag: False
Metadata: {}

=== Incoming Edges (Messages TO worker_1) ===
Sources: ['supervisor']

=== Outgoing Edges (Messages FROM worker_1) ===
Destinations: []

=== All Conversations Involving worker_1 ===
Messages from supervisor to worker_1: 1
  [2025-10-26 06:23:21.365701+00:00] supervisor -> worker_1: Come up with 3 ideas for a new product....

Messages from worker_1 to supervisor: 1
  [2025-10-26 06:23:21.365701+00:00] supervisor -> worker_1: Come up with 3 ideas for a new product....

=== Control Relationships ===
Controllers of worker_1: ['supervisor']
Subordinates of worker_1: []
